In [3]:
!pip install flask pyngrok pillow
!pip install opencv-python-headless

In [4]:
import io
import base64
import numpy as np
import tensorflow as tf
from flask import Flask, request, jsonify
from PIL import Image
from pyngrok import ngrok
from google.colab import drive
import cv2
import getpass

In [5]:

app = Flask(__name__)



drive.mount('/content/drive')



CAMINHO_MODELO = '/content/drive/My Drive/projeto_cnh/modelo_cnh_final.keras'

modelo = tf.keras.models.load_model(CAMINHO_MODELO)



@app.route('/prever', methods=['POST'])

def prever():

    try:

        dados = request.get_json()

        if not dados or 'image' not in dados:

            return jsonify({"erro": "O campo image é obrigatório"}), 400
        base64_string = dados['image']

        if "," in base64_string:

            base64_string = base64_string.split(",")[1]

        img_data = base64.b64decode(base64_string)
        img = Image.open(io.BytesIO(img_data)).convert('RGB')
        img = img.resize((254, 254))

        img_array = np.array(img).astype(np.float32)
        img_array = np.expand_dims(img_array, axis=0)

        predicao = modelo.predict(img_array, verbose=0)

        valor_bruto = float(predicao[0][0])

        e_cnh = valor_bruto < 0.5

        confianca = float(1 - valor_bruto if e_cnh else valor_bruto)

        return jsonify(
              {
            "status": "sucesso",
            "confianca": f"{(confianca * 100):.2f}%",
            "e_uma_cnh": "Sim, é uma CNH" if e_cnh else "Não é uma CNH",
        })
    except Exception as e:
        return jsonify({"status": "erro", "mensagem": str(e)}), 500

token = input("Digite seu token aqui")
ngrok.set_auth_token(token)
public_url = ngrok.connect(5000)
print(f"URL: {public_url.public_url}/prever")

app.run()

Mounted at /content/drive
Digite seu token aqui38qrMFAb7SwMRdJiYtlB8ovF974_4oopVWcfZMe63wZQbmJ1P
URL: https://braxton-agleam-laically.ngrok-free.dev/prever
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
